In [4]:
# Install necessary libraries (if not already installed)
!pip install numpy pandas obspy matplotlib librosa tensorflow

In [ ]:
import os
import numpy as np
import pandas as pd
from obspy import read
from scipy.signal import butter, filtfilt, decimate
from datetime import timedelta
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import joblib

# Function to eliminate power frequency noise
def eliminate_power_frequency(signal, fs, power_freqs=[50, 60], bandwidth=2):
    nyquist = 0.5 * fs
    cleaned_signal = signal.copy()

    for power_freq in power_freqs:
        if power_freq < nyquist:
            low = max(0.1, (power_freq - bandwidth / 2)) / nyquist
            high = min(0.99, (power_freq + bandwidth / 2) / nyquist
)
            b, a = butter(2, [low, high], btype='bandstop')
            cleaned_signal = filtfilt(b, a, cleaned_signal)
    return cleaned_signal

# Define an enhanced CNN-LSTM model in PyTorch
class CNNLSTMModel(nn.Module):
    def __init__(self):
        super(CNNLSTMModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.lstm = nn.LSTM(input_size=64, hidden_size=32, num_layers=2, batch_first=True)
        self.fc = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = x.permute(0, 2, 1)  # LSTM expects (batch, seq_len, features)
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # Take the output from the last time step
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

# Train a CNN-LSTM model (for demonstration purposes)
def train_cnn_lstm_model(data_dir):
    sequences = []
    labels = []
    catalogs = [os.path.join(data_dir, 'mars/training/catalogs/Mars_InSight_training_catalog.csv'),
                os.path.join(data_dir, 'lunar/training/catalogs/apollo12_catalog_GradeA_final.csv')]

    for catalog in catalogs:
        catalog_df = pd.read_csv(catalog)
        for _, row in catalog_df.iterrows():
            if 'lunar' in catalog:
                filepath = os.path.join(data_dir, 'lunar', 'training', 'data', 'S12_GradeA', f"{row['filename']}.mseed")
            else:
                filepath = os.path.join(data_dir, 'mars', 'training', 'data', f"{row['filename']}")
            if os.path.exists(filepath):
                label = 1 if row['mq_type'].strip() == 'impact_mq' else 0
                st = read(filepath)
                tr = st[0]
                tr_data = eliminate_power_frequency(tr.data, tr.stats.sampling_rate)
                tr_data = decimate(tr_data, 4)  # Downsample the data to reduce sequence length
                sequences.append(tr_data)
                labels.append(label)
            else:
                print(f"Warning: File {filepath} not found. Skipping.")

    if not sequences:
        print("No valid sequences found for training. Please verify the catalog paths and file availability.")
        return

    # Pad sequences to the same length
    max_length = max([len(seq) for seq in sequences])
    sequences_padded = np.array([np.pad(seq, (0, max_length - len(seq)), 'constant') for seq in sequences])
    labels = np.array(labels)

    # Normalize the data
    scaler = StandardScaler()
    sequences_scaled = scaler.fit_transform(sequences_padded)

    # Reshape data for CNN-LSTM input
    sequences_scaled = sequences_scaled[:, np.newaxis, :]

    # Convert data to PyTorch tensors
    X = torch.tensor(sequences_scaled, dtype=torch.float32)
    y = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)

    # Define model, loss function, and optimizer
    model = CNNLSTMModel()
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Move model to MPS device if available
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model.to(device)

    # Training loop with batching
    batch_size = 16  # Define a batch size to reduce memory usage
    num_epochs = 50
    num_batches = len(X) // batch_size + (1 if len(X) % batch_size != 0 else 0)

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        for i in range(num_batches):
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, len(X))
            X_batch = X[start_idx:end_idx].to(device)
            y_batch = y[start_idx:end_idx].to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/num_batches:.4f}')

    # Save the trained model and scaler
    torch.save(model.state_dict(), 'seismic_event_cnn_lstm.pth', _use_new_zipfile_serialization=False)
    joblib.dump(scaler, 'scaler.pkl')

# Train the model before running the analysis
train_cnn_lstm_model('data')

# Main analysis function
def analysis(filepath):
    # Load trained model and scaler
    model = CNNLSTMModel()
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model.to(device)
    try:
        model.load_state_dict(torch.load('seismic_event_cnn_lstm.pth', map_location=device))
        model.eval()
        scaler = joblib.load('scaler.pkl')
    except FileNotFoundError:
        print("Model or scaler not found. Please train the model first using train_cnn_lstm_model().")
        return

    # Read in seismic data using ObsPy
    st = read(filepath)

    # Extract the original data
    tr_original = st[0]  # Extract the trace
    tr_times_original = tr_original.times()
    tr_data_original = tr_original.data

    # Apply bandpass filter for cleaned data
    minfreq = 0.2
    maxfreq = 2.0
    tr_filt = st.copy().filter('bandpass', freqmin=minfreq, freqmax=maxfreq)[0]
    tr_times_filt = tr_filt.times()
    tr_data_filt = tr_filt.data

    # Eliminate power frequencies (50Hz, 60Hz)
    sampling_rate = tr_filt.stats.sampling_rate
    tr_data_filt_cleaned = eliminate_power_frequency(tr_data_filt, sampling_rate)
    tr_data_filt_cleaned = decimate(tr_data_filt_cleaned, 4)  # Downsample the data

    # Pad or truncate the data to match training length
    max_length = scaler.n_features_in_
    if len(tr_data_filt_cleaned) < max_length:
        tr_data_filt_cleaned = np.pad(tr_data_filt_cleaned, (0, max_length - len(tr_data_filt_cleaned)), 'constant')
    else:
        tr_data_filt_cleaned = tr_data_filt_cleaned[:max_length]

    # Prepare data for model prediction
    tr_data_filt_cleaned = scaler.transform(tr_data_filt_cleaned.reshape(1, -1))
    tr_data_filt_cleaned = torch.tensor(tr_data_filt_cleaned[:, np.newaxis, :], dtype=torch.float32).to(device)

    # Predict using the trained model
    with torch.no_grad():
        prediction = model(tr_data_filt_cleaned).item()
    
    if prediction < 0.5:
        print("No seismic event detected.")
        return
    else:
        print("Seismic event detected.")

    # Correct the trigger line placement
    max_vel_time = tr_times_filt[np.argmax(tr_data_filt)]
    best_trigger_time_abs = tr_filt.stats.starttime + timedelta(seconds=max_vel_time)
    best_trigger_time_str = best_trigger_time_abs.strftime('%Y-%m-%dT%H:%M:%S.%f')

    print(f"Trigger Time: {best_trigger_time_str} at relative time {max_vel_time}")

    # Plot the graph for the **filtered** data with the Best Trigger On line
    decimation_factor = 4
    tr_times_filt_decimated = tr_times_filt[::decimation_factor]

    plt.figure(figsize=(12, 6))
    plt.plot(tr_times_filt_decimated, tr_data_filt_cleaned.cpu().numpy().flatten(), label="Filtered Data")
    plt.axvline(x=max_vel_time, color='green', label='Best Trigger On')
    plt.xlim([min(tr_times_filt_decimated), max(tr_times_filt_decimated)])
    plt.ylabel('Velocity (m/s)')
    plt.xlabel('Time (s)')
    plt.title(f'{filepath} - Filtered Data', fontweight='bold')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Plot the graph for the **original** data with the Best Trigger On line
    plt.figure(figsize=(12, 6))
    plt.plot(tr_times_original, tr_data_original, label="Original Data")
    plt.axvline(x=max_vel_time, color='green', label='Best Trigger On')
    plt.xlim([min(tr_times_original), max(tr_times_original)])
    plt.ylabel('Velocity (m/s)')
